In [ ]:
%%writefile bubble_merge_sort.cpp
#include <stdio.h>
#include <stdlib.h>
#include <omp.h>

#define SIZE 1000

void swap(int *a, int *b) {
    int temp = *a; *a = *b; *b = temp;
}

void parallelBubbleSort(int arr[], int n) {
    for (int i = 0; i < n; i++) {
        #pragma omp parallel for
        for (int j = i % 2; j < n - 1; j += 2)
            if (arr[j] > arr[j + 1])
                swap(&arr[j], &arr[j + 1]);
    }
}

void merge(int arr[], int l, int m, int r) {
    int n1 = m - l + 1, n2 = r - m;
    int *L = (int *)malloc(n1 * sizeof(int));
    int *R = (int *)malloc(n2 * sizeof(int));
    for (int i = 0; i < n1; i++) L[i] = arr[l + i];
    for (int i = 0; i < n2; i++) R[i] = arr[m + 1 + i];

    int i = 0, j = 0, k = l;
    while (i < n1 && j < n2)
        arr[k++] = (L[i] <= R[j]) ? L[i++] : R[j++];
    while (i < n1) arr[k++] = L[i++];
    while (j < n2) arr[k++] = R[j++];
    free(L); free(R);
}

void parallelMergeSort(int arr[], int l, int r) {
    if (l < r) {
        int m = (l + r) / 2;
        #pragma omp parallel sections
        {
            #pragma omp section
            parallelMergeSort(arr, l, m);
            #pragma omp section
            parallelMergeSort(arr, m + 1, r);
        }
        merge(arr, l, m, r);
    }
}

void fillArray(int arr[], int size) {
    for (int i = 0; i < size; i++) arr[i] = rand() % 1000;
}

int main() {
    int arr1[SIZE], arr2[SIZE];
    fillArray(arr1, SIZE);
    for (int i = 0; i < SIZE; i++) arr2[i] = arr1[i];

    double start = omp_get_wtime();
    parallelBubbleSort(arr1, SIZE);
    printf("Parallel Bubble Sort Time: %f seconds\n", omp_get_wtime() - start);

    start = omp_get_wtime();
    parallelMergeSort(arr2, 0, SIZE - 1);
    printf("Parallel Merge Sort Time: %f seconds\n", omp_get_wtime() - start);

    return 0;
}

In [ ]:
!g++ -fopenmp bubble_merge_sort.cpp -o Sort

In [ ]:
!./Sort

Parallel Bubble Sort Time: 0.006775 seconds
Parallel Merge Sort Time: 0.000835 seconds
